In [161]:
cd 'drive/MyDrive/Twitter Multi Class (Cleaned Dataset)/'

[Errno 2] No such file or directory: 'drive/MyDrive/Twitter Multi Class (Cleaned Dataset)/'
/content/drive/MyDrive/Twitter Multi Class (Cleaned Dataset)


In [162]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub

# !pip install tf-models-official
!pip install -q tf-models-official==2.4.0
!pip install flask-ngrok

from official import nlp
from official.nlp import bert
import official.nlp.bert.tokenization as tokenization


import tweepy
import time
import warnings
warnings.filterwarnings('ignore')

import flask
from flask import Flask,request,render_template
from flask_ngrok import run_with_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [163]:
!pip install pyngrok==4.1.1
!ngrok authtoken 27Pje6OgHm5TTondBCn2qbMxIqu_6JK8rditdSHNYZFKnEj9X

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [164]:
#model path
model_fname = 'Twitter Multi-Class Sentiment Classification Final(Cleaned Dataset)'
my_wd = '/content/drive/MyDrive/Twitter Multi Class (Cleaned Dataset)/'

In [165]:
#twitter cinfigurations
consumer_key="QJNiunDxSNkv80F3uOvmU6Xuv"
consumer_secret="DSQKMBu5YG0neTumwyWqpGJ5y9s3Vc0EEfkiNQ7aSH2PZWDwyz"
access_token="1502003594178232322-4bSpwQ2V6qig9tDEm6nqKiJnXOCkhl"
access_token_secret="L57d23UQawkj7auUC6j6V6jnD3XPGKMq7zqiue1KamaYw"

##Accessing the Twitter API

In [166]:
auth=tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token=(access_token, access_token_secret)
api= tweepy.API(auth, wait_on_rate_limit= True)

##Tokenization Process

In [167]:
tokenizer = bert.tokenization.FullTokenizer(
    vocab_file=os.path.join(my_wd, model_fname, 'assets/vocab.txt'),
    do_lower_case=False)

In [168]:
max_seq_length = 512

def encode_names(n, tokenizer):
   tokens = list(tokenizer.tokenize(n))
   tokens.append('[SEP]')
   return tokenizer.convert_tokens_to_ids(tokens)

def bert_encode(string_list, tokenizer, max_seq_length):
  num_examples = len(string_list)
  
  string_tokens = tf.ragged.constant([
      encode_names(n, tokenizer) for n in np.array(string_list)])

  cls = [tokenizer.convert_tokens_to_ids(['[CLS]'])]*string_tokens.shape[0]
  input_word_ids = tf.concat([cls, string_tokens], axis=-1)

  input_mask = tf.ones_like(input_word_ids).to_tensor(shape=(None, max_seq_length))

  type_cls = tf.zeros_like(cls)
  type_tokens = tf.ones_like(string_tokens)
  input_type_ids = tf.concat(
      [type_cls, type_tokens], axis=-1).to_tensor(shape=(None, max_seq_length))

  inputs = {
      'input_word_ids': input_word_ids.to_tensor(shape=(None, max_seq_length)),
      'input_mask': input_mask,
      'input_type_ids': input_type_ids}

  return inputs



1. The code defines a function to encode a list of names into a list of ids.

2. The function takes a list of strings and a tokenizer as input.

3. The function encodes the list of strings into a list of ids.

4. The function uses the TF ragged library to create a list of ids that correspond to the examples in the list of strings.

5. The function creates a list of class ids and token ids using the tokenizer.

6. The function creates a tensor that represents the input word ids.

7. The function creates a tensor that represents the input mask.

8. The function creates a tensor that represents the input type ids.

9. The function merges the class ids, token ids, and input word ids into a single tensor.

10. The function returns the merged tensor.

##Load saved model

In [169]:
#load saved model
saved_model = tf.keras.models.load_model(os.path.join(my_wd, model_fname))

##Functions to Calculate Sentiment

In [170]:
def passData(path):
    filter = "-filter:retweets"
    text_query = path + filter
    count=300
    try:
        #creation of query method using parameters
        tweets= tweepy.Cursor(api.search, q=text_query, tweet_mode="extended", lang='en').items(count)

        #getting the information from twitter object
        tweet_list= [[tweet.full_text] for tweet in tweets]
        #creating a data frame from the list
        tweet_df= pd.DataFrame(tweet_list, columns=['Content'])
            
    except BaseException as e:
        print('failed_on_status', str(e))
        time.sleep(3)

        

# 1. The code creates a query method that uses parameters.

# 2. The code retrieves tweets based on the text_query parameter.

# 3. The code retrieves the information from the tweets retrieved in step 2.

# 4. The code creates a data frame from the list of tweets retrieved in step 3.

    df2 = tweet_df.replace('@','',regex = True).replace(',','',regex = True).replace('#','',regex = True).replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace('RT','',regex = True)
    df2.drop_duplicates(subset ="Content",keep = False, inplace = True)
    
    tweetsList = []

    for index, row in df2.iterrows():
      tweet = str(row["Content"])
      tweetsList.append(tweet)

    if(len(tweetsList) >20):
      n = 50
      result = calculate(tweetsList[:n])
    else:
      n = 10
      result = calculate(tweetsList[:n])

    response = {}
    response = {
          "sentence": (tweetsList[:n]),
          "sentiment":(result[0]),
          "negative_count":(result[1]),
          "neutral_count":(result[2]),
          "positive_count":(result[3]),
          "overall_sentiment":(result[4])
      }
      

# The code first calculates the sentiment of the first 10 tweets in the list.
#  If the list has more than 20 tweets, then it calculates the sentiment of the first 50 tweets. 
# It then stores the sentiment, negative count, neutral count, positive count, and overall sentiment in a response dictionary.
    return response

In [171]:
def calculate(tweets):
  inputs = bert_encode(string_list=list(tweets),
                    tokenizer=tokenizer, 
                    max_seq_length=512)

  prediction = saved_model.predict(inputs)


# 1. The code imports the bert_encode module, which is used to encode a list of tweets into a format that the machine learning model can understand.

# 2. The tokenizer module is used to break the tweets into individual words.

# 3. The max_seq_length parameter is set to 512, which limits the length of the encoded tweets to 512 characters or less.

# 4. The encoded tweets are then passed to the machine learning model for prediction.
  results = []
  positive_count = 0
  negative_count = 0
  neutral_count = 0
  overall_sentiment = 0                                                       

# 1. The code sets up some variables including a count of positive, negative, and neutral sentiment examples found in the text.

# 2. It then goes through each prediction in the text and determines the sentiment based on the most likely outcome.

# 3. The code then keeps track of the total number of positive, negative, and neutral sentiment examples found.

# 4. It then calculates the overall sentiment based on the positive, negative, and neutral sentiment examples found.

  for y in prediction:
      if(np.argmax(y)== 0):
        result = 0
        results.append(result)
        negative_count += 1
      elif(np.argmax(y)== 1):
        result = 1
        results.append(result)
        neutral_count += 1
      elif(np.argmax(y)== 2):
        result = 2
        results.append(result)
        positive_count += 1
      else:
        result = "Didnt get the expected result"
        results.append(result)

  if(positive_count > negative_count) and (positive_count > neutral_count):
    overall_sentiment = "Positive"
  elif(negative_count > positive_count) and (negative_count > neutral_count):
    overall_sentiment = "Negative"
  else:
    overall_sentiment = "Neutral"

  return results,negative_count,neutral_count,positive_count,overall_sentiment

##Flask App

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)


@app.route("/")
def index():
  return render_template("index.html")


@app.route("/calculate", methods = ["POST"])
def predict():
  path = request.form['search']

  response = passData(path)

  return render_template("calculate.html", value = response)
  


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://9d77-34-143-190-230.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [20/Jul/2022 21:03:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/images/icons8-clapperboard-36(-hdpi).png HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/images/icons8-clapperboard-24(-mdpi).png HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/images/BERT-logo-new.png HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:49] "GET /static/images/Sentiment%20analysis-amico.png HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:50] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:50] "GET /static/images/icons8-clapperboard-36(-hdpi).png HTTP/1.1" 200 -
127.0.0.1 - - [20/Jul/2022 21:03:50] "GET /sta



1. The code imports the Flask web application framework.

2. The code uses the Flask framework to run a web application.

3. The code defines a route for the web application to respond to requests for the URL /.

4. The code defines a route for the web application to respond to requests for the URL /calculate.

5. The code defines a function called predict that takes a path argument.

6. The code uses the request.form dictionary to extract the value of the search argument from the POST request.

7. The code calls the passData function with the path argument.

8. The code uses the render_template function to return a template named calculate.html.

9. The code returns the value of the response variable to the caller.